In [3]:
# ! pip install pandas==1.5.3
# ! pip install numpy==1.23.1
from importlib import reload
import pandas as pd 

In [41]:
from bloomberg.ai.librarian import Librarian, get_config
from bloomberg.ai.librarian.newsarchive.functions import has_wire_usage_criteria

config = get_config()
config.spark.properties['spark.executor.instances'] = 20
config.spark.properties['spark.executor.memory'] = '12G'
config.spark.properties['spark.driver.memory'] = '16G'

librarian = Librarian(config)
spark = librarian.spark_session
ds = librarian.datasets.newsarchive.prod.v2_0()
newsarchive_df = (
    ds
        .add_partition('pvf_newsldap_4')
        .add_partition('pvf_newsldap_6')
        .begin(year=2023, month=1, day=1)
        .end(year=2023, month=6, day=10)
        .fetch()
)

# Filter by Wire Usage Criteria
criteria = {'derivative_product': True}
newsarchive_df = newsarchive_df.filter(has_wire_usage_criteria(**criteria))

In [4]:
nana_df = pd.read_csv('big-processed-nana-file-0.csv.gz', compression='gzip', index_col=0)

In [17]:
nana_spark_df = spark.createDataFrame(nana_df[['story_suid']]) 

/layers/com.bloomberg.ds.buildpacks.spark/spark/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [27]:
newsarchive_df.suid == nana_spark_df.story_suid

Column<b'(suid = story_suid)'>

In [49]:
newsarchive_df.columns

['suid',
 'wire',
 'class',
 'bloom_lang',
 'headline',
 'hotlevel',
 'indicator_bitmap',
 'legacy_txtencoding',
 'pub_date',
 'story_state',
 'timeofarrival',
 'updatetime',
 'version_type',
 'deleteflag',
 'is_priv_freeish',
 'ptype',
 'pbits',
 'pay_per_view',
 'brdirect',
 'allow_headline_url_on_feed',
 'alt_ptype',
 'alt_pbits',
 'use_wire_pbit',
 'wire_mnemonic',
 'buyside',
 'dj',
 'wire_name',
 'deleted',
 'press_release',
 'bbnews',
 'bmr',
 'allow_body_on_feed',
 'embargoed',
 'is_priv_not_for_clients',
 'nse_release',
 'platts_wire',
 'is_web',
 'wire_pbit',
 'restricted',
 'is_serve_from_web',
 'body',
 'body_state',
 'body_texttype',
 'footer',
 'slug',
 'nicodes',
 'people',
 'tickers',
 'story_attachment',
 'bunch_id',
 'delete_flag',
 'suid_role',
 'story_roles_updatetime',
 'waiting_for_role',
 'agentid',
 'agentname',
 'url_part1',
 'url_part2',
 'url_part3',
 'url_part4',
 'derived_nicodes',
 'derived_people',
 'derived_tickers',
 'groupid']

In [71]:
# newsarchive_df.filter(newsarchive_df.bbnews==True).select('body').head(5)
# newsarchive_df.select('is_serve_from_web').show()

In [78]:
combined_df = (
    newsarchive_df
        .select('suid', 'headline', 'hotlevel', 'buyside', 'is_serve_from_web', 'body',)
        .join(nana_spark_df, newsarchive_df.suid == nana_spark_df.story_suid)
        .toPandas()
)

In [81]:
combined_df.head()

,suid,headline,hotlevel,buyside,is_serve_from_web,body,story_suid
0,RTRCRJMS385E,Takkt 1Q Ebitda EU30.2M Vs. EU32.7M Y/y,0,False,False,"<sections>\n<section class=""news-rsf-text-head...",RTRCRJMS385E
1,RTRUO9MRXLVL,Hertz 1Q Adjusted EPS Beats Estimates,0,False,False,"<sections>\n<section class=""news-rsf-text-head...",RTRUO9MRXLVL
2,RQ25T3DWX2PS,"Telkom Expects Swiftnet Offers in March, Recei...",0,False,False,"<sections>\n<section class=""news-rsf-text-head...",RQ25T3DWX2PS
3,RUUBOH6RV5DT,Burberry FY Retail Sales Meets Estimates,0,False,False,"<sections>\n<section class=""news-rsf-text-head...",RUUBOH6RV5DT
4,RUUBLE6RV5DV,Premier Foods FY Adjusted Pretax Profit Beats ...,0,False,False,"<sections>\n<section class=""news-rsf-text-head...",RUUBLE6RV5DV
